In [2]:
import torch
import requests
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import json
import re
from typing import List, Dict, Tuple
import nltk
# from rouge_score import rouge_scorer
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import os
from google.colab import files
import base64
from io import BytesIO

# Download NLTK data
nltk.download('punkt', quiet=True)

print("✅ Setup complete!")

✅ Setup complete!


In [4]:
training_data = [
            {
                "image_path":"/content/baked_pasta.jpg",
                "title":"cheesy bake",
                "full_instructions":"""Preheat oven to 375°F. Cook 1 lb penne pasta according to package directions. In a large skillet, brown 1 lb ground beef with diced onions. Add marinara sauce and simmer. Mix pasta with meat sauce, transfer to baking dish. Top with mozzarella and parmesan cheese. Cover with foil and bake 25 minutes. Remove foil and bake 10 more minutes until cheese is golden.""",
                "summary":"1. Cook pasta and brown ground beef with onions and marinara sauce. 2. Mix pasta with sauce, top with cheese in baking dish. 3. Bake covered at 375°F for 25 min, then uncovered 10 min until golden.",

            },
            {
                "image_path":"/content/stir_fry.jpg",
                "title":"noodly thing",
                "full_instructions":"""Heat oil in wok over high heat. Add sliced chicken breast and cook 3-4 minutes. Add mixed vegetables (bell peppers, carrots, snap peas) and stir-fry 2 minutes. Add cooked rice noodles, soy sauce, oyster sauce, and sesame oil. Toss everything together for 2-3 minutes until heated through. Garnish with green onions and sesame seeds.""",
                "summary":"1. Stir-fry chicken in hot oil, then add mixed vegetables. 2. Add cooked rice noodles and sauces (soy, oyster, sesame). 3. Toss everything together until heated, garnish with green onions.",

            },
            {
                "image_path":"/content/chocolate_cake.jpg",
                "title":"dark sweet thing",
                "full_instructions":"""Preheat oven to 350°F. Mix flour, cocoa powder, baking soda, and salt. In separate bowl, cream butter and sugar, add eggs and vanilla. Alternately add dry ingredients and buttermilk to wet mixture. Pour into greased pans and bake 30-35 minutes. Cool completely. Make frosting by beating butter, powdered sugar, cocoa, and cream. Frost cooled cake layers.""",
                "summary":"1. Mix dry ingredients separately from creamed butter, sugar, eggs, and buttermilk. 2. Combine wet and dry ingredients, bake in greased pans at 350°F for 30-35 min. 3. Cool completely and frost with chocolate buttercream.",

            },
            {
                "image_path":"/content/soup.jpg",
                "title":"warm bowl stuff",
                "full_instructions":"""Heat olive oil in large pot. Sauté diced onions, carrots, and celery until soft, about 5 minutes. Add minced garlic and cook 1 minute. Pour in chicken broth and bring to boil. Add diced potatoes and simmer 15 minutes. Add cooked chicken pieces and corn. Season with salt, pepper, and herbs. Simmer 5 more minutes until potatoes are tender.""",
                "summary":"1. Sauté onions, carrots, celery, and garlic in olive oil until soft. 2. Add chicken broth, potatoes, and simmer 15 minutes. 3. Add chicken and corn, season and simmer 5 more minutes.",

            },
            {
                "image_path":"/content/tacos.jpg",
                "title":"folded stuff",
                "full_instructions":"""Season ground beef with cumin, chili powder, garlic powder, and salt. Brown meat in skillet over medium heat, breaking it up as it cooks, about 8 minutes. Warm corn tortillas in dry pan or microwave. Fill tortillas with seasoned beef. Top with diced onions, cilantro, lime juice, and salsa. Serve with lime wedges and hot sauce on the side.""",
                "summary":"1. Brown seasoned ground beef in skillet until cooked through. 2. Warm corn tortillas and fill with seasoned meat. 3. Top with onions, cilantro, lime juice, and salsa.",

            },
            {
                "image_path":"/content/pizza,jpg.webp",
                "title":"round flat bread",
                "full_instructions":"""Preheat oven to 475°F. Roll out pizza dough on floured surface to desired thickness. Transfer to greased pizza pan or baking sheet. Spread pizza sauce evenly, leaving 1-inch border for crust. Sprinkle with mozzarella cheese and desired toppings like pepperoni, mushrooms, peppers. Bake 12-15 minutes until crust is golden and cheese is bubbly and slightly browned.""",
                "summary":"1. Roll out dough and spread with sauce, leaving border for crust. 2. Add mozzarella cheese and desired toppings. 3. Bake at 475°F for 12-15 minutes until golden and bubbly.",

            },
            {
                "image_path":"/content/salad.jpg",
                "title":"green mix",
                "full_instructions":"""Wash and dry mixed greens thoroughly. Chop cucumber, tomatoes, and red onion into bite-sized pieces. In small bowl, whisk together olive oil, balsamic vinegar, Dijon mustard, salt, and pepper for dressing. Place greens in large bowl, add chopped vegetables. Drizzle with dressing and toss gently to coat. Top with crumbled feta cheese and serve immediately.""",
                "summary":"1. Wash greens and chop vegetables (cucumber, tomatoes, onion). 2. Whisk together olive oil, balsamic vinegar, mustard, salt, and pepper. 3. Toss greens and vegetables with dressing, top with feta cheese.",

            },
            {
                "image_path":"/content/pancake.jpg",
                "title":"fluffy rounds",
                "full_instructions":"""In large bowl, whisk together flour, sugar, baking powder, and salt. In separate bowl, beat eggs, then add milk, melted butter, and vanilla. Pour wet ingredients into dry ingredients and stir just until combined - don't overmix. Heat griddle or pan over medium heat and lightly grease. Pour 1/4 cup batter for each pancake. Cook until bubbles form on surface, then flip and cook until golden brown.""",
                "summary":"1. Mix dry ingredients separately from wet ingredients (eggs, milk, butter, vanilla). 2. Combine wet and dry ingredients gently without overmixing. 3. Cook on greased griddle until bubbles form, flip, and cook until golden.",

            },
            {
                "image_path":"/content/curry.jpg",
                "title":"orange sauce thing",
                "full_instructions":"""Heat oil in large pan. Add diced onions and cook until translucent. Add minced garlic, ginger, and curry powder, cook 1 minute until fragrant. Add diced tomatoes and cook until soft. Pour in coconut milk and bring to simmer. Add cubed chicken and simmer 15 minutes until cooked through. Add frozen peas in last 5 minutes. Season with salt and serve over rice.""",
                "summary":"1. Sauté onions, then add garlic, ginger, and curry powder until fragrant. 2. Add tomatoes and coconut milk, bring to simmer. 3. Add chicken and cook 15 minutes, add peas in last 5 minutes, serve over rice.",

            },
            {
                "image_path":"/content/sandwich.jpg",
                "title":"stacked bread",
                "full_instructions":"""Toast bread slices until golden brown. Spread mayonnaise on one side of each slice. Layer lettuce leaves, sliced tomatoes, cooked bacon strips, and sliced turkey on bottom slice. Add slice of cheese if desired. Season tomatoes with salt and pepper. Top with second slice of bread, mayo side down. Cut diagonally and secure with toothpicks if needed.""",
                "summary":"1. Toast bread and spread with mayonnaise. 2. Layer lettuce, tomatoes, bacon, and turkey on bottom slice. 3. Season, top with second slice, cut diagonally.",

            }

           ]

print(f"Training dataset : {len(training_data)} samples")

Training dataset : 10 samples


In [5]:
test_data = [
             {
                "image_path":"/content/meatball.jpg",
                "title":"round meat things",
                "full_instructions":"""Combine ground beef, breadcrumbs, egg, minced garlic, parmesan cheese, salt, and pepper in bowl. Mix gently with hands until just combined. Roll mixture into 1.5-inch meatballs. Heat oil in large skillet over medium-high heat. Brown meatballs on all sides, about 8 minutes total. Add marinara sauce to pan, reduce heat to low, cover and simmer 15 minutes until cooked through.""",

            },
            {
                "image_path":"/content/smoothie.jpg",
                "title":"thick drink",
                "full_instructions":"""Add frozen berries, banana, Greek yogurt, honey, and milk to blender. Blend on high speed for 60-90 seconds until completely smooth and creamy. Stop and scrape down sides if needed. Taste and adjust sweetness with more honey if desired. Add more milk if too thick, or more frozen fruit if too thin. Pour into glasses and serve immediately with straws.""",

            },
            {
                "image_path":"/content/cookies.jpg",
                "title":"sweet rounds",
                "full_instructions":"""Preheat oven to 375°F. Cream butter and sugars in large bowl until fluffy. Beat in eggs and vanilla. In separate bowl, whisk flour, baking soda, and salt. Gradually mix dry ingredients into wet mixture. Stir in chocolate chips. Drop rounded tablespoons of dough onto ungreased baking sheets, spacing 2 inches apart. Bake 9-11 minutes until edges are golden but centers still soft.""",

            },
            {
                "image_path":"/content/risotto.jpg",
                "title":"creamy rice",
                "full_instructions":"""Heat chicken broth in saucepan and keep warm. In large pan, sauté diced onions in butter until translucent. Add arborio rice and stir 2 minutes until coated. Add white wine and stir until absorbed. Add warm broth one ladle at a time, stirring constantly, waiting for each addition to be absorbed before adding more. Continue for 18-20 minutes until rice is creamy. Stir in parmesan cheese and butter.""",

            },
            {
                "image_path":"/content/grilled_chicken.jpg",
                "title":"cooked bird",
                "full_instructions":"""Marinate chicken breasts in olive oil, lemon juice, garlic, herbs, salt, and pepper for at least 30 minutes. Preheat grill to medium-high heat. Remove chicken from marinade and grill 6-7 minutes per side, or until internal temperature reaches 165°F. Let rest 5 minutes before slicing. Serve with grilled vegetables or salad. Can also be cooked in grill pan on stovetop if outdoor grill not available.""",

            }
           ]

print(f"Test dataset : {len(test_data)} samples")

Test dataset : 5 samples


In [6]:
model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

print("model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model loaded successfully!


In [7]:
class RecipeSummarizationPipeline:
    def __init__(self, model, processor, training_examples):
        self.model = model
        self.processor = processor
        self.training_examples = training_examples

    def create_few_shot_prompt(self, title: str, instructions: str, num_examples: int = 3) -> str:
        """few-shot prompt with training examples"""

        #examples for few-shot learning
        examples = self.training_examples[:num_examples]

        prompt = """You are an expert chef assistant. Your task is to create concise 2-3 step cooking summaries from detailed instructions.

Look at these examples:

"""

        #few-shot examples
        for i, example in enumerate(examples, 1):
            prompt += f"""Example {i}:
Title: "{example['title']}"
Full Instructions: {example['full_instructions']}
Summary: {example['summary']}

"""

        prompt += f"""Now, create a summary for this new recipe:

Title: "{title}"
Full Instructions: {instructions}

Generate a concise 2-3 step summary that captures the essential cooking process. Each step should be clear and actionable.

Summary:"""

        return prompt

    def load_image(self, image_path: str) -> Image.Image:
        try:
            if image_path.startswith(('http://', 'https://')):
                response = requests.get(image_path)
                image = Image.open(BytesIO(response.content))
            else:
                image = Image.open(image_path)

            image = image.convert('RGB')

            max_size = 512
            if max(image.size) > max_size:
                image.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)

            return image
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            #dummy image if loading fails
            return Image.new('RGB', (224, 224), color='white')

    def generate_summary(self, image_path: str, title: str, full_instructions: str) -> str:
        """Generate cooking summary using LLaVA"""

        # Load image
        image = self.load_image(image_path)

        # Create few-shot prompt
        text_prompt = self.create_few_shot_prompt(title, full_instructions)

        # inputs = self.processor(text_prompt, image, return_tensors="pt").to(self.model.device)
        # Prepare inputs with proper conversation format for LLaVA
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": text_prompt}
                ]
            }
        ]

        # Apply chat template
        prompt = self.processor.apply_chat_template(conversation, add_generation_prompt=True)

        # Process inputs
        inputs = self.processor(images=image, text=prompt, return_tensors="pt")

        # Move to device
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        # Generate response
        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=self.processor.tokenizer.eos_token_id
            )

        # Decode response
        response = self.processor.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

        # Extract summary from response
        summary = self.extract_summary(response)
        return summary

    def extract_summary(self, response: str) -> str:
        """extract summary from model response"""
        if "Summary:" in response:
            summary = response.split("Summary:")[-1].strip()
        else:
            summary = response.strip()

        summary = re.sub(r'^["\']|["\']$', '', summary)
        summary = summary.strip()

        if len(summary) < 10:
            return "Unable to generate summary from the given information."

        return summary

pipeline = RecipeSummarizationPipeline(model, processor, training_data)



In [8]:
results = []

for i, test_sample in enumerate(test_data, 1):
    print(f"\nProcessing Test Sample {i}")
    print(f"Title: {test_sample['title']}")
    print(f"Image: {test_sample['image_path']}")

    try:
        # Generate summary
        generated_summary = pipeline.generate_summary(
            test_sample['image_path'],
            test_sample['title'],
            test_sample['full_instructions']
        )

        print(f"\nGenerated Summary:")
        print(generated_summary)


        results.append({
            'test_sample': test_sample,
            'generated_summary': generated_summary,

        })

    except Exception as e:
        print(f"Error processing sample {i}: {str(e)}")
        results.append({
            'test_sample': test_sample,
            'generated_summary': f"Error: {str(e)}",

        })


print("\nSummary generation complete!")



Processing Test Sample 1
Title: round meat things
Image: /content/meatball.jpg

Generated Summary:
1. Combine ground beef, breadcrumbs, egg, minced garlic, parmesan cheese, salt, and pepper in a bowl. Mix gently with hands until just combined.
2. Roll mixture into 1.5-inch meatballs.
3. Heat oil in a large skillet over medium-high heat, and brown the meatballs on all sides, about 8 minutes total.
4. Add marinara sauce to the pan, reduce heat to low, cover and simmer for 15 minutes until cooked through.

Processing Test Sample 2
Title: thick drink
Image: /content/smoothie.jpg

Generated Summary:
1. Combine frozen berries, banana, Greek yogurt, honey, and milk in a blender.
2. Blend on high speed for 60-90 seconds until smooth and creamy.
3. Taste and adjust sweetness and thickness as needed, then pour into glasses and serve immediately with straws.

Processing Test Sample 3
Title: sweet rounds
Image: /content/cookies.jpg

Generated Summary:
1. Cream butter and sugars, beat in eggs and 